<a href="https://colab.research.google.com/github/anushka-code/Clima-Flutter/blob/master/Code_Smell_Classification_using_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Code Smell Detection (Smelly/Non Smelly) using Structural & Semantic Features 

#Code Smells Targetted: 


1.   Long Parameters List
2.   Switch Statements







###Importing Libraries

In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 38.6 MB/s 
     |████████████████████████████████| 77 kB 6.9 MB/s 
     |████████████████████████████████| 895 kB 40.9 MB/s 
     |████████████████████████████████| 6.6 MB 37.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [98]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Functional, Model
from keras.layers import Input, Convolution1D, MaxPooling1D, Flatten, Dense, concatenate
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Embedding, Bidirectional
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn import model_selection
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import imblearn
from imblearn.over_sampling import SMOTE
import nltk 
from nltk.tokenize import RegexpTokenizer
import re
import gensim
from gensim.models import Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec
import transformers
from transformers import TFDistilBertModel, DistilBertConfig, DistilBertTokenizerFast




# import transformers
# from transformers import DistilBertTokenizer
# from transformers import TFDistilBertModel

###Mounting Google Drive

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

###Dataset Loader

In [4]:
def DataLoader(link, name_of_file):
  id = link.split("/")[-2]
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile(name_of_file)
  dataframe = pd.read_csv(name_of_file)
  return dataframe

link1 = 'https://drive.google.com/file/d/1EfbAqgr7i9h4yFwEoU3igG34Gt48l6WT/view?usp=sharing'
link2 = 'https://drive.google.com/file/d/1Ya1OMWsz1yyXAaZheIck-roX0M9UWiqg/view?usp=sharing'
link3 = 'https://drive.google.com/file/d/1xeMRB6ACyjep6VF_rDRq8cXkA58ZlEZs/view?usp=sharing'
link4 = 'https://drive.google.com/file/d/1OxxMX7-kc7mQUvb2DSJR_fbtvZ5bB1Ef/view?usp=sharing'
link5 = 'https://drive.google.com/file/d/1mRIgoXhLwCdao7f9yD2QTSi7JNZNq9zJ/view?usp=sharing'
link6 = 'https://drive.google.com/file/d/1rLkJAwHkBAAkHMp2L1Y1AzuniIZdyB7x/view?usp=sharing'
name1 = 'long_parameters_list_structural.csv'
name2 = 'switch_statements_structural.csv'
name3 = 'long_parameters_list_semantic.csv'
name4 = 'switch_statements_semantic.csv'
name5 = 'multilabel_classification_dataset.csv'
name6 = 'semantic_final.csv'

df_lp = DataLoader(link1, name1)
df_ss = DataLoader(link2, name2)
df_lpm = DataLoader(link3, name3)
df_ssm = DataLoader(link4, name4)
df_multilabel = DataLoader(link5, name5)
df_semantic = DataLoader(link6,name6)

Long Parameters List Structural Dataset

In [5]:
df_lp.head(5)

,NOP_method,CC_method,ATFD_method,FDP_method,CM_method,MAXNESTING_method,LOC_method,CYCLO_method,NMCS_method,NOLV_method,...,NOI_package,LOC_package,NOM_package,NOPK_project,NOCS_project,NOI_project,NOM_project,NOMNAMM_project,LOC_project,is_long_parameters_list
0,0,0,0,0,0,2,8,2,0,0,...,0,229,16,43,849,78,5788,4778,65687,False
1,0,0,0,0,0,1,3,1,0,0,...,0,4223,240,43,849,78,5788,4778,65687,False
2,1,0,0,0,0,1,2,1,0,1,...,0,4223,240,43,849,78,5788,4778,65687,False
3,0,5,0,0,9,0,1,1,0,0,...,11,6037,728,43,849,78,5788,4778,65687,False
4,1,0,1,1,0,1,3,1,0,1,...,11,6037,728,43,849,78,5788,4778,65687,False


Switch Statements Structural Dataset



In [6]:
df_ss.head(5)

,NOP_method,CC_method,ATFD_method,FDP_method,CM_method,MAXNESTING_method,LOC_method,CYCLO_method,NMCS_method,NOLV_method,...,NOI_package,LOC_package,NOM_package,NOPK_project,NOCS_project,NOI_project,NOM_project,NOMNAMM_project,LOC_project,is_switch_statements
0,1,2,0,0,2,1,5,1,0,1,...,3,439,53,43,849,78,5788,4778,65687,False
1,0,0,0,0,0,2,6,3,0,0,...,0,4223,240,43,849,78,5788,4778,65687,False
2,2,0,0,0,0,1,3,1,0,2,...,0,4223,240,43,849,78,5788,4778,65687,False
3,0,0,0,0,0,2,3,1,0,0,...,0,1212,120,43,849,78,5788,4778,65687,False
4,0,0,0,0,0,1,3,1,0,0,...,0,516,29,43,849,78,5788,4778,65687,False


Long Parameters List Semantic Dataset

In [7]:
df_lpm.head(5)

,project,package,class,method,is_code_smell
0,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes.common.plugins,ReminderHookBase,shutdownMapHook(),False
1,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes.mindmapmode,MindMapMapModel.StringReaderCreator,createReader(),False
2,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes.mindmapmode,MindMapController,newModel(ModeController),False
3,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes,MindMapNode,getColor(),False
4,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes,ControllerAdapter.ControllerPopupMenuListener,popupMenuCanceled(PopupMenuEvent),False


Switch Statements Semantic Dataset

In [8]:
df_ssm.head(5)

,project,package,class,method,is_code_smell
0,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes.mindmapmode.actions.xml,AbstractXmlAction,actionPerformed(ActionEvent),False
1,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes.mindmapmode,EncryptedMindMapNode,childrenUnfolded(),False
2,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes.mindmapmode,freemind.modes.mindmapmode.MindMapController$1,"apply(MindMapMapModel,MindMapNodeModel)",False
3,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.view.mindmapview.attributeview,freemind.view.mindmapview.attributeview.Attrib...,run(),False
4,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes.mindmapmode.listeners,MindMapNodeMotionListener,stopDrag(),False


### Data Pre-Processing of Structural Dataset


In [9]:
def PrePro(last_column, dataframe):
  dataframe.rename(columns = {last_column :'is_code_smell'}, inplace = True) #rename column
  dataframe['is_code_smell'] = dataframe["is_code_smell"].astype(int) #change boolean labels to int labels
  Y_part = dataframe.iloc[:,-1:]
  X_part = dataframe.iloc[:,:56]
  X_part = X_part.replace(to_replace =["?"], value = np.nan) #replace non existing values with null
  X_part = X_part.astype(float) #change datatype of features of X as float
  return X_part,Y_part

X_lp, Y_lp = PrePro('is_long_parameters_list',df_lp)
X_ss, Y_ss = PrePro('is_switch_statements',df_ss)

# X_m, Y_m = PrePro(df_multilabel)

In [10]:
def MeanforNaN(dataframe):   #function to fill null spaces with column mean 
  column_means = dataframe.mean()
  dataframe = dataframe.fillna(column_means)
  return dataframe

X_lp = MeanforNaN(X_lp)
X_ss = MeanforNaN(X_ss)

In [11]:
def ConCat(df1,df2): #concatenate code smell datasets
  code_smells = [df1,df2]
  joint = pd.concat(code_smells)
  return joint

X_train = ConCat(X_lp,X_ss)
Y_train = ConCat(Y_lp,Y_ss)

###Data Normalization

In [12]:
def Normalize(dataframe): #apply MinMax normalisation to fit the values between 0 to 1
  scaler = MinMaxScaler()
  model = scaler.fit(dataframe)
  scaled_data = model.transform(dataframe)
  return scaled_data

# X_m_sample = Normalize(X_m)
X_sample = Normalize(X_train)

###Synthetic Minority Oversampling Technique (SMOTE) Algo for Imbalanced Datasets


In [13]:
Y_train.value_counts() #1/3rd are smelly, 2/3rd are non smelly 

is_code_smell
0                573
1                267
dtype: int64

In [14]:
Y_sample = Y_train.to_numpy(dtype='int64', copy='True')

In [15]:
def Oversample(X_data,Y_data): # Using Smote obtain a 50-50 balanced dataset 

  sm = SMOTE(random_state = 2)
  X_train_res, Y_train_res = sm.fit_resample(X_data, Y_data.ravel())
  return X_train_res, Y_train_res

X_new, Y_new = Oversample(X_sample,Y_sample)

In [16]:
X_train_final = X_new.reshape((1146,56,1)) # hence makes 573 smelly and 573 non smelly samples

In [17]:
# X_struct = pd.DataFrame(X_new)

In [18]:
Y_train_final = Y_new.reshape(1146,)

###Word Embeddings using Gensim Word2Vec

In [19]:
# def WordEmbedding(tokenizedwords): #creates a dictionary with unique vocab words and corresponding vectors 
#   model = Word2Vec(tokenizedwords, min_count=1, sg=1, size=50) #skipgram model 
#   model.train(tokenizedwords, total_examples=model.corpus_count, epochs=10)
#   keys = list(model.wv.vocab.keys())
#   values = list(model.wv.vectors)
#   wdict = {}
#   for i in range(840):
#     wdict[keys[i]] = values[i]
#   return wdict

# word_vec = WordEmbedding(word_groups) #this dictionary contains the key as word and value as the vector

In [20]:
# def VectorRep(groupofwords,dictionary): #gives a list of list which contains vectors for each word 
#   rep=[]
#   for i in groupofwords:
#     l=[]
#     for j in i:
#       l.append(dictionary.get(j))
#     rep.append(l)
#   return rep

# vector_rep = VectorRep(word_groups, word_vec)

In [21]:
# def AvgLength(vectorrepresentation):
#   sum = 0
#   for i in range(840):
#     sum += len(vectorrepresentation[i])
#   avg = sum/840
#   return avg

# AvgLength(vector_rep)

In [22]:
# def FindMaxLength(lst):
#     maxList = max((x) for x in lst)
#     maxLength = max(len(x) for x in lst )
  
#     return maxList, maxLength
      
# print(FindMaxLength(word_groups))

In [23]:
# x = np.zeros(50)
# print(x)
# type(x)
# m = x.tolist()
# type(m)

In [24]:
# tensor1 = tf.zeros(4,4)
# print(tensor1)

In [25]:
# tensor = tf.convert_to_tensor(vector_rep[0]) 
# print(tensor.shape)

In [26]:
# type(vector_rep)

###Text Pre Processing

In [27]:
# df_sem = ConCat(df_lpm,df_ssm) #join long parameters list and switch statements dataset

In [28]:
def PrePro2(dataframe):
  X_part = dataframe.iloc[:,:-1]  
  return X_part

X_sem= PrePro2(df_semantic) #separate as X and Y columns

In [29]:
def ToList(dataframe, string): #converts pandas.core.series.Series to list of lists
  new_column = dataframe[string].tolist()
  return list(map(lambda x:[x], new_column))

class_words = ToList(X_sem, 'class')
method_words = ToList(X_sem, 'method')

###Tokenization

In [30]:
def TokenizeGroups(text): #tokenizes as per CamelCase RegEx and converts to lowercase
  tokenizer = RegexpTokenizer('[a-zA-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))') # RegEx pattern for CamelCase 
  useful_text = tokenizer.tokenize(text) 
  useful_text = [x.lower() for x in useful_text]
  return useful_text

In [31]:
def LoopOver(list): #loops over all the samples to tokenize all the strings in each word group
  for i in range(0,1146):
    list[i] = TokenizeGroups(list[i][0])
    i=i+1
  return list

class_words_sem = LoopOver(class_words)
method_words_sem = LoopOver(method_words)

In [32]:
def WordList(list1,list2): #concatenate the words of each project, package, class and method
  final_list = []
  for i in range(0,1146):
    x = list1[i] + list2[i]
    final_list.append(x)
  res = [' '.join(ele) for ele in final_list]
  return res

word_groups = WordList(class_words_sem, method_words_sem)

In [33]:
# def Sentences(group_of_words): #makes a list of lists containing sentences for each single sample 
#   lst = []
#   for i in range(0,1146):
#     appendlist = []
#     string = group_of_words[i]
#     appendstr = ' '.join(string)
#     appendlist.append(appendstr)
#     lst.append(appendlist)
#   return lst

# sentence_groups = Sentences(word_groups)

In [34]:
token = Tokenizer()
token.fit_on_texts(word_groups)
vocab_size = len(token.word_index) + 1
print(vocab_size)

1300


In [35]:
encoded_text = token.texts_to_sequences(word_groups)

In [36]:
max_length = 176
X = pad_sequences(encoded_text, maxlen = max_length, padding = 'pre')
print(X)

[[  0   0   0 ... 841  12 239]
 [  0   0   0 ... 511  46 165]
 [  0   0   0 ...  51 166 106]
 ...
 [  0   0   0 ...   1   1   1]
 [  0   0   0 ...   1   1   2]
 [  0   0   0 ...   2   2   2]]


In [37]:
# token.word_index.items()

In [38]:
# X.shape

###Glove Vectors

In [39]:
# import gensim.downloader as api
# glove_gensim  = api.load('glove-wiki-gigaword-100') #100 dimension

In [40]:
# vector_size = 100 
# gensim_weight_matrix = np.zeros(( vocab_size ,vector_size)) 
# gensim_weight_matrix.shape

In [41]:
# for word, index in token.word_index.items():
#     if index < vocab_size: 
#         if word in glove_gensim.wv.vocab:
#             gensim_weight_matrix[index] = glove_gensim[word]
#         else:
#             gensim_weight_matrix[index] = np.zeros(100)

###Stratified K-Fold Cross Validation for Limited Data


In [42]:
# def CrossValidation(dataX,dataY): #split the dataset into train and test using k-5 fold cross validation
#   skf = StratifiedKFold(n_splits=5, random_state = None, shuffle=True) 

#   for train_index, test_index in skf.split(dataX , dataY):
#         X_crosstrain, X_crosstest = dataX[train_index], dataX[test_index] 
#         Y_crosstrain, Y_crosstest = dataY[train_index], dataY[test_index]
#         return X_crosstrain,X_crosstest,Y_crosstrain,Y_crosstest

# X_crosstrain,X_crosstest,Y_crosstrain,Y_crosstest = CrossValidation(X_train_final, Y_train_final)

### Model 1 - Machine Learning Pipeline of Feature Union with RF Classifier, TF-IDF, GridSearchCV

In [43]:
# X_final = pd.concat([X_struct, word_groups], axis=1) # combining the structural and semantic data
# X_ml_final = X_final.to_numpy() # converting df into numpy array

In [44]:
# def CrossValidation(dataX,dataY): #split the dataset into train and test using k-5 fold cross validation
#   skf = StratifiedKFold(n_splits=5, random_state = None, shuffle=True) 

#   for train_index, test_index in skf.split(dataX , dataY):
#         X_crosstrain, X_crosstest = dataX[train_index], dataX[test_index] 
#         Y_crosstrain, Y_crosstest = dataY[train_index], dataY[test_index]
#         return X_crosstrain,X_crosstest,Y_crosstrain,Y_crosstest

# X_crosstrain,X_crosstest,Y_crosstrain,Y_crosstest = CrossValidation(X_ml_final, Y_train_final)

In [45]:
# def get_numeric_data(x):
#     return [record[:-2].astype(float) for record in x] #gets the numeric data which is all the columns except last column

# def get_text_data(x):
#     return [record[-1] for record in x] # gets the text data which is the last column of the dataset

# transformer_numeric = FunctionTransformer(get_numeric_data) #transformer_numeric when transformed on some X will apply get_numeric_data on X
# transformer_text = FunctionTransformer(get_text_data)

In [46]:
# pipeline = Pipeline([
#     ('features', FeatureUnion([
#             ('numeric_features', Pipeline([
#                 ('selector', transformer_numeric)
#             ])),
#              ('text_features', Pipeline([
#                 ('selector', transformer_text),
#                 ('vec', TfidfVectorizer(analyzer='word'))
#             ]))
#          ])),
#     ('clf', RandomForestClassifier())
# ])

# param_grid = {'clf__n_estimators': np.linspace(1, 100, 42, dtype=int),
#               'clf__min_samples_split': [2],
#               'clf__min_samples_leaf': [1],
#               'clf__max_features': [8],
#               'clf__max_depth': [None],
#               'clf__criterion': ['entropy'],
#               'clf__bootstrap': [False]}

# scoring = {'Accuracy': 'accuracy', 'F1': 'f1_macro'}
# refit = 'F1'

In [47]:
# def ML_codesmell(X_tr, Y_tr, X_ts, Y_ts):
#   rf_model = GridSearchCV(pipeline, param_grid=param_grid, scoring=scoring,refit=refit, n_jobs=-1, return_train_score=True, verbose=1)
#   rf_model.fit(X_tr, Y_tr)
#   model_score = rf_model.score(X_ts, Y_ts)
#   y_pred_test = rf_model.predict(X_ts)
#   accuracy = accuracy_score(Y_ts, y_pred_test)
#   classification = classification_report(Y_ts, y_pred_test)
#   return model_score, accuracy, classification

# model_score, accuracy_score, classification_report = ML_codesmell(X_crosstrain, Y_crosstrain, X_crosstest, Y_crosstest)

In [48]:
# print("Model Score : ", model_score)
# print("Accuracy Score : ", accuracy_score)
# print("Classification Report : ", classification_report)

### Model 2 - Multimodal of CNN & BiLSTM

In [ ]:
# height, width, depth = 1146, 56, 1
# input_shape=(width,depth)
# input_struct = Input(shape=input_shape)
# layer_1 = Convolution1D(filters=32, kernel_size=3, activation='relu')(input_struct)
# layer_2 = Convolution1D(filters=64, kernel_size=3, activation='relu')(layer_1)
# layer_3 = Convolution1D(filters=64, kernel_size=3, activation='relu')(layer_2)
# flatten_cnn = Flatten()(layer_3)

# model_left = Model(input_struct, flatten_cnn)
# #dense_cnn = Dense(500, activation='relu')(flatten)

In [ ]:
# EMBEDDING_DIM = 100
# input_sem = Input(shape=(176))
# embedding_layer = Embedding(input_dim = vocab_size, output_dim = EMBEDDING_DIM, input_length= X.shape[1],
#                     weights = [gensim_weight_matrix],trainable = False)(input_sem)
# layer_1 = Bidirectional(LSTM(100,return_sequences=True))(embedding_layer)
# layer_2 = Bidirectional(LSTM(200,return_sequences=True))(layer_1)
# layer_3 = Bidirectional(LSTM(100,return_sequences=False))(layer_2)
# flatten_bilstm = Flatten()(layer_3)

# model_right = Model(input_sem, flatten_bilstm)
# #dense_bilstm = Dense(500, activation='relu')(layer_3)

In [ ]:
# merge_layer = concatenate([flatten_cnn, flatten_bilstm])
# dense_1 = Dense(500, activation='relu')(merge_layer)
# dense_2 = Dense(100, activation='relu')(dense_1)
# dense3 = Dense(20, activation='relu')(dense_2)
# dense_final = Dense(1, activation='sigmoid')(dense3)

In [ ]:
# merged_model = Model([input_struct, input_sem], dense_final)

In [ ]:
# merged_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# merged_model.fit([X_train_final, X], y=Y_train_final, batch_size=100, epochs=10, validation_split = 0.2)

In [ ]:
# input_sem = Input(shape=(1,))
# embedding = embedding_layer(input_sem)
# bilstm1 = Bidirectional(CuDNNLSTM(100,return_sequences=True))(embedding)
# bilstm2 = Bidirectional(CuDNNLSTM(200,return_sequences=True))(bilstm1)
# bilstm3 = Bidirectional(CuDNNLSTM(200,return_sequences=True))(bilstm2)
# bilstm4 = Bidirectional(CuDNNLSTM(100,return_sequences=False))(bilstm3)

# model_right.add(Embedding(input_dim = vocab_size, output_dim = EMBEDDING_DIM, input_length= X.shape[1],
#                     weights = [gensim_weight_matrix],trainable = False))
# model_right.add(Bidirectional(CuDNNLSTM(100,return_sequences=True)))
# model_right.add(Bidirectional(CuDNNLSTM(200,return_sequences=True)))
# model_right.add(Bidirectional(CuDNNLSTM(200,return_sequences=True)))
# model_right.add(Bidirectional(CuDNNLSTM(100,return_sequences=False)))
# model_right.add(Flatten())

In [ ]:
# model_left = Sequential()
# model_left.add(Convolution1D(filters=32, kernel_size=3, activation='relu', input_shape=(56,1)))
# model_left.add(Convolution1D(filters=64, kernel_size=3, activation='relu'))
# model_left.add(Convolution1D(filters=64, kernel_size=3, activation='relu'))
# model_left.add(Flatten())

### Model 3 - distilBERT pretrained + CNN

In [49]:
max = 0
for ele in word_groups:
  res = len(ele.split())
  if res > max:
    max = res

print(max)

28


In [110]:
params = {'MAX_LENGTH': 30,
          'EPOCHS': 50,
          'LEARNING_RATE': 5e-5,
          'FT_EPOCHS': 5,
          'OPTIMIZER': 'adam',
          'FT_LEARNING_RATE': 2e-5,
          'BATCH_SIZE': 64,
          'NUM_STEPS': len(X_train.index) // 64,
          'DISTILBERT_DROPOUT': 0.2,
          'DISTILBERT_ATT_DROPOUT': 0.2,
          'LAYER_DROPOUT': 0.2,
          'KERNEL_INITIALIZER': 'GlorotNormal',
          'BIAS_INITIALIZER': 'zeros',
          'POS_PROBA_THRESHOLD': 0.5,          
          'ADDED_LAYERS': 'Dense 256, Dense 32, Dropout 0.2',
          'LR_SCHEDULE': '5e-5 for 6 epochs, Fine-tune w/ adam for 6 epochs @2e-5',
          'FREEZING': 'All DistilBERT layers frozen for 6 epochs, then unfrozen for 6',
          'CALLBACKS': '[early_stopping monitoring val_loss w/ patience=0]',
          'RANDOM_STATE':42
          }

In [111]:
def batch_encode(tokenizer, texts, batch_size=191, max_length=params['MAX_LENGTH']):
    input_ids = []
    attention_mask = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer.batch_encode_plus(batch,
                                             max_length=max_length,
                                             padding='longest',
                                             truncation=True,
                                             return_attention_mask=True,
                                             return_token_type_ids=False
                                             )
        input_ids.extend(inputs['input_ids'])
        attention_mask.extend(inputs['attention_mask'])
    
    
    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_mask)

In [113]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [114]:
X_train_ids, X_train_attention = batch_encode(tokenizer, word_groups)

In [115]:
X_train_ids

<tf.Tensor: shape=(1146, 30), dtype=int32, numpy=
array([[  101, 14764,  8103, ...,     0,     0,     0],
       [  101,  2568,  4949, ...,     0,     0,     0],
       [  101,  2568,  4949, ...,     0,     0,     0],
       ...,
       [  101,  4026,  2131, ...,     0,     0,     0],
       [  101,  4026,  2131, ...,     0,     0,     0],
       [  101,  4026,  2131, ...,     0,     0,     0]], dtype=int32)>

In [116]:
distilBERT = TFDistilBertModel.from_pretrained('distilbert-base-uncased', output_hidden_states = True)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_projector', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [117]:
for layer in distilBERT.layers:
    layer.trainable = False

In [58]:
# weight_initializer = tf.keras.initializers.GlorotNormal(seed=params['RANDOM_STATE'])    

In [59]:
# input_ids_layer = tf.keras.layers.Input(shape=(30,), 
#                                             name='input_ids', 
#                                             dtype='int32')

In [60]:
# input_ids_layer

In [61]:
# input_attention_layer = tf.keras.layers.Input(shape=(30,), 
#                                                   name='input_attention', 
#                                                   dtype='int32')

In [62]:
# input_attention_layer

In [118]:
output = distilBERT([X_train_ids, X_train_attention])

In [119]:
hidden_states = output[1]

In [120]:
tfsentence = []
sentence_embeddings = []
sentence_emb_arr = []

In [121]:
for i in range(1146):
  tfsentence.append(hidden_states[-2][i])

In [122]:
for i in range(1146):
  samplei = tf.math.reduce_mean(tfsentence[i], axis=0)
  sentence_embeddings.append(samplei)

In [123]:
len(sentence_embeddings)

1146

In [124]:
for i in range(1146):
  storei = sentence_embeddings[i]
  storearr = storei.numpy()
  sentence_emb_arr.append(storearr)

In [125]:
numarr = np.array(sentence_emb_arr)

In [126]:
numarr.shape

(1146, 768)

In [127]:
concat = np.concatenate((numarr, X_new), axis=1)

In [128]:
X_bert = concat.reshape(1146,824,1)

In [130]:
from keras.models import Sequential

In [132]:
model = Sequential()
model.add(Convolution1D(filters=32, kernel_size=3, activation='relu', input_shape=(824,1)))
model.add(Convolution1D(filters=64, kernel_size=3, activation='relu'))
model.add(Convolution1D(filters=64, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 822, 32)           128       
                                                                 
 conv1d_4 (Conv1D)           (None, 820, 64)           6208      
                                                                 
 conv1d_5 (Conv1D)           (None, 818, 64)           12352     
                                                                 
 flatten_1 (Flatten)         (None, 52352)             0         
                                                                 
 dense_4 (Dense)             (None, 100)               5235300   
                                                                 
 dense_5 (Dense)             (None, 50)                5050      
                                                                 
 dense_6 (Dense)             (None, 25)               

In [133]:
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

In [136]:
model.fit(X_bert,Y_train_final, batch_size=100, epochs=20, validation_split=0.25)

Epoch 1/20
9/9 [==============================] - 0s 44ms/step - loss: 0.0259 - accuracy: 0.9895 - val_loss: 0.4451 - val_accuracy: 0.8920
Epoch 2/20
9/9 [==============================] - 0s 40ms/step - loss: 0.0335 - accuracy: 0.9860 - val_loss: 0.2805 - val_accuracy: 0.9373
Epoch 3/20
9/9 [==============================] - 0s 33ms/step - loss: 0.0280 - accuracy: 0.9860 - val_loss: 0.3961 - val_accuracy: 0.9129
Epoch 4/20
9/9 [==============================] - 0s 32ms/step - loss: 0.0342 - accuracy: 0.9884 - val_loss: 0.3480 - val_accuracy: 0.9164
Epoch 5/20
9/9 [==============================] - 0s 32ms/step - loss: 0.0362 - accuracy: 0.9872 - val_loss: 0.3457 - val_accuracy: 0.9373
Epoch 6/20
9/9 [==============================] - 0s 32ms/step - loss: 0.0298 - accuracy: 0.9884 - val_loss: 0.3272 - val_accuracy: 0.9303
Epoch 7/20
9/9 [==============================] - 0s 33ms/step - loss: 0.0259 - accuracy: 0.9907 - val_loss: 0.3456 - val_accuracy: 0.9164
Epoch 8/20
9/9 [===========

In [ ]:
# model.fit(X_crosstrain,Y_crosstrain, batch_size=50, epochs=150)

In [ ]:
# X1 = tf.keras.layers.Dense(256,
#                               activation='relu',
#                               kernel_initializer=weight_initializer,
#                               bias_initializer='zeros'
#                               )(cls_token)

In [ ]:
# X1

In [ ]:
# nn_meta = X_new

In [ ]:
# concat = tf.keras.layers.Concatenate(axis=1)([X1,nn_meta])

In [ ]:
# concat

In [ ]:
# nn_meta = X_new

# def build_model(transformer, max_length=params['MAX_LENGTH']):

#     weight_initializer = tf.keras.initializers.GlorotNormal(seed=params['RANDOM_STATE'])    

#     input_ids_layer = tf.keras.layers.Input(shape=(max_length,), 
#                                             name='input_ids', 
#                                             dtype='int32')
    
#     input_attention_layer = tf.keras.layers.Input(shape=(max_length,), 
#                                                   name='input_attention', 
#                                                   dtype='int32')
    
#     last_hidden_state = transformer([input_ids_layer, input_attention_layer])[0]
    
#     cls_token = last_hidden_state[:, 0, :]
    
#     X1 = tf.keras.layers.Dense(256,
#                               activation='relu',
#                               kernel_initializer=weight_initializer,
#                               bias_initializer='zeros'
#                               )(cls_token)
    
#     concat = tf.keras.layers.Concatenate(axis=1)([X1,nn_meta])
    
#     X2 = tf.keras.layers.Dense(128,
#                               activation='relu',
#                               kernel_initializer=weight_initializer,
#                               bias_initializer='zeros'
#                               )(concat)
    
#     X3 = tf.keras.layers.Dropout(params['LAYER_DROPOUT'],
#                                  seed=params['RANDOM_STATE']
#                                 )(X2)

#     X4 = tf.keras.layers.Dense(32,
#                               activation='relu',
#                               kernel_initializer=weight_initializer,
#                               bias_initializer='zeros'
#                               )(X3)

#     output = tf.keras.layers.Dense(1, 
#                                    activation='sigmoid',
#                                    kernel_initializer=weight_initializer,
#                                    bias_initializer='zeros'
#                                    )(X4)

#     model = tf.keras.Model([input_ids_layer, input_attention_layer], output)
    
#     model.compile(tf.keras.optimizers.Adam(learning_rate=params['LEARNING_RATE']), 
#                   loss='binary_crossentropy',
#                   metrics=['accuracy'])
    
#     return model

In [ ]:
# model = build_model(distilBERT)

In [ ]:
# from keras import backend as K

# def recall_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     recall = true_positives / (possible_positives + K.epsilon())
#     return recall

# def precision_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     return precision

# def f1_m(y_true, y_pred):
#     precision = precision_m(y_true, y_pred)
#     recall = recall_m(y_true, y_pred)
#     return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# optimizer = tf.keras.optimizers.Adam(learning_rate=params['FT_LEARNING_RATE'])

# model.compile(optimizer=optimizer, 
#               loss='binary_crossentropy',
#               metrics=['accuracy', f1_m,precision_m, recall_m])

# # early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
# #                                                   mode='min',
# #                                                   min_delta=0,
# #                                                   patience=0,
# #                                                   restore_best_weights=True)

# train_history2 = model.fit(
#     x = [X_train_ids, X_train_attention],
#     y = Y_train_final,
#     epochs = params['FT_EPOCHS'],
#     batch_size = params['BATCH_SIZE'],
#     steps_per_epoch = params['NUM_STEPS'],
#     validation_split = 0.3,
#     # callbacks=[early_stopping],
# )

### Bi-LSTM

In [ ]:
# EMBEDDING_DIM = 100
# class_num = 1
# model = Sequential()
# model.add(Embedding(input_dim = vocab_size, output_dim = EMBEDDING_DIM, input_length= X.shape[1],
#                     weights = [gensim_weight_matrix],trainable = False))
# model.add(Bidirectional(CuDNNLSTM(100,return_sequences=True)))
# model.add(Bidirectional(CuDNNLSTM(200,return_sequences=True)))
# model.add(Bidirectional(CuDNNLSTM(200,return_sequences=True)))
# model.add(Bidirectional(CuDNNLSTM(100,return_sequences=False)))
# model.add(Dense(1, activation = 'sigmoid'))
# model.compile(optimizer = 'adam', loss = 'binary_crossentropy',  metrics = ['accuracy'])
# model.summary()

In [ ]:
# model.fit(X, Y_train_final , epochs = 50, batch_size = 100)

In [ ]:
# def Sentences(group_of_words): #makes a list of lists containing sentences for each single sample 
#   lst = []
#   for i in range(0,840):
#     appendlist = []
#     string = group_of_words[i]
#     appendstr = ' '.join(string)
#     appendlist.append(appendstr)
#     lst.append(appendlist)
#   return lst

# concat_word_groups = Sentences(word_groups)

### CNN 




In [ ]:
# model = Sequential()
# model.add(Convolution1D(filters=32, kernel_size=3, activation='relu', input_shape=(56,1)))
# model.add(Convolution1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Convolution1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Flatten())
# model.add(Dense(100, activation='relu'))
# model.add(Dense(50, activation='relu'))
# model.add(Dense(25, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

In [ ]:
# model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# model.fit(X_crosstrain,Y_crosstrain, batch_size=50, epochs=150)
# accuracy = model.evaluate(X_crosstest, Y_crosstest, batch_size=50)
# print(accuracy)

In [ ]:
# Y_tensor =tf.convert_to_tensor(Y_tr)

In [ ]:
# height, width, depth = X_tr.shape[0], X_tr.shape[1], X_tr.shape[2]
# input_shape=(width,depth)
# input = Input(shape=input_shape)
# X = Convolution1D(filters=32, kernel_size=3, activation='relu')(input)
# X = Convolution1D(filters=64, kernel_size=3, activation='relu')(X)
# X = Convolution1D(filters=64, kernel_size=3, activation='relu')(X)
# X = Flatten()(X)
# concatenate = tf.keras.layers.Concatenate(axis=1)([X, berttensor1])
# X = Dense(100, activation='relu')(concatenate)
# X = Dense(50, activation='relu')(X)
# X = Dense(10, activation = 'relu')(X)
# output = Dense(1, activation = "sigmoid")(X)

# model = Model(inputs=input, outputs=output)

In [ ]:
# newinput = Input(shape=[10880])
# hidden1 = Dense(20, activation='relu')(newinput)
# hidden2 = Dense(10, activation='relu')(hidden1)
# output = Dense(1, activation='sigmoid')(hidden2)
# model = Model(inputs=newinput, outputs=output)

In [ ]:
# epochs, batch_size, test_batch_size = 50, 50, 50
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(X_tr,Y_tr, batch_size=batch_size, epochs=epochs)

In [ ]:
# def evaluate_model(trainX, trainy, testX, testy): 
# 	epochs, batch_size, test_batch_size = 120, 50, 50
# 	height, width, depth = trainX.shape[0], trainX.shape[1], trainX.shape[2]
# 	model = Sequential()
# 	model.add(Convolution1D(filters=32, kernel_size=3, activation='relu', input_shape=(width,depth)))
# 	model.add(Convolution1D(filters=64, kernel_size=3, activation='relu'))
# 	model.add(Flatten())
# 	model.add(Dense(1, activation='sigmoid'))
# 	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# 	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size)
# 	_, accuracy = model.evaluate(testX, testy, batch_size=test_batch_size)
# 	return accuracy

# model_accuracy = evaluate_model(X_crosstrain, Y_crosstrain, X_crosstest, Y_crosstest)